In [26]:
import pandas as pd
import os
import sqlite3
import re
import html

## 1. Data

In [4]:
os.chdir("../blueprints-text/")

In [5]:
data_path = "data"

In [28]:
from packages.blueprints.exploration import count_words;

### 1.1 Get the data

In [7]:
posts_df = pd.read_csv(os.path.join(data_path, "reddit-selfposts/rspct_autos.tsv.gz"), sep="\t")

In [8]:
posts_df

,id,subreddit,title,selftext
0,8f73s7,Harley,No Club Colors,Funny story. I went to college in Las Vegas. T...
1,5s0q8r,Mustang,Roush vs Shleby GT500,"I am trying to determine which is faster, and ..."
2,5z3405,Volkswagen,2001 Golf Wagon looking for some insight,Hello! <lb><lb>Trying to find some information...
3,7df18v,Lexus,IS 250 Coolant Flush/Change,https://www.cars.com/articles/how-often-should...
4,5tpve8,volt,Gen1 mpg w/ dead battery?,"Hi, new to this subreddit. I'm considering bu..."
...,...,...,...,...
19995,7i2k6y,4Runner,Bilstein Shocks,I read a lot Forums and people recommend getti...
19996,83p2kv,Harley,Question on potential purchase of crashed bike.,I am thinking about buying a 2010 Harley Spor...
19997,7x722h,volt,Got our first warning light on our dash,My husband and I were headed somewhere and I w...
19998,7v2xmg,Lexus,Any IS models to avoid?,I am looking at getting a used Lexus IS (2014 ...


In [9]:
subred_df = pd.read_csv(os.path.join(data_path, "reddit-selfposts/subreddit_info.csv.gz")).set_index(["subreddit"])

In [10]:
subred_df

,category_1,category_2,category_3,in_data,reason_for_exclusion
subreddit,,,,,
whatsthatbook,advice/question,book,NaN,True,NaN
CasualConversation,advice/question,broad,NaN,False,too_broad
Clairvoyantreadings,advice/question,broad,NaN,False,too_broad
DecidingToBeBetter,advice/question,broad,NaN,False,too_broad
HelpMeFind,advice/question,broad,NaN,False,too_broad
...,...,...,...,...,...
HFY,writing/stories,sci-fi,NaN,True,NaN
TalesFromYourServer,writing/stories,tech support,NaN,False,fewer posts than r/talesfromtechsupport which ...
talesfromtechsupport,writing/stories,tech support,NaN,True,NaN


In [11]:
df = pd.merge(posts_df, subred_df, on="subreddit")

In [12]:
df

,id,subreddit,title,selftext,category_1,category_2,category_3,in_data,reason_for_exclusion
0,8f73s7,Harley,No Club Colors,Funny story. I went to college in Las Vegas. T...,autos,harley davidson,NaN,True,NaN
1,4z6vee,Harley,Mystery Rattle,I have a stock 2010 FXD that has a rate I can'...,autos,harley davidson,NaN,True,NaN
2,6x4tdk,Harley,Gum on tailpipe,So I'm on a bike trip with my father and we st...,autos,harley davidson,NaN,True,NaN
3,7l7rer,Harley,Changing up the bars!,So I’m gonna change out my T bars that came on...,autos,harley davidson,NaN,True,NaN
4,683pc7,Harley,Spent 2 days and made some Mini Floorboards fo...,"A few weeks ago, somebody was looking for some...",autos,harley davidson,NaN,True,NaN
...,...,...,...,...,...,...,...,...,...
19995,7346dx,4Runner,03 JBL Sound System Issue,Just picked up my first 4Runner this week! Lov...,autos,toyota,NaN,True,NaN
19996,5ut9e2,4Runner,2nd gen Comes Back From the Dead With No Tools,Hello fellow runners-<lb><lb>I wanted to share...,autos,toyota,NaN,True,NaN
19997,66xnwd,4Runner,Tranny cooler in cold temperatures?,So I picked up a 2002 4Runner recently and the...,autos,toyota,NaN,True,NaN
19998,8kvxkg,4Runner,Torn between a Wrangler and a 4 Runner...,Ok so I've posted to r/Jeep and figured I shou...,autos,toyota,NaN,True,NaN


In [13]:
column_mapping = {
    'id': 'id',
    'subreddit': 'subreddit',
    'title': 'title',
    'selftext': 'text',
    'category_1': 'category',
    'category_2': 'subcategory',
    'category_3': None,
    'in_data': None,
    'reason_for_exclusion': None
}

In [14]:
columns = [c for c in column_mapping.keys() if column_mapping[c] != None]
columns

['id', 'subreddit', 'title', 'selftext', 'category_1', 'category_2']

In [15]:
df = df[columns].rename(columns=column_mapping)
df

,id,subreddit,title,text,category,subcategory
0,8f73s7,Harley,No Club Colors,Funny story. I went to college in Las Vegas. T...,autos,harley davidson
1,4z6vee,Harley,Mystery Rattle,I have a stock 2010 FXD that has a rate I can'...,autos,harley davidson
2,6x4tdk,Harley,Gum on tailpipe,So I'm on a bike trip with my father and we st...,autos,harley davidson
3,7l7rer,Harley,Changing up the bars!,So I’m gonna change out my T bars that came on...,autos,harley davidson
4,683pc7,Harley,Spent 2 days and made some Mini Floorboards fo...,"A few weeks ago, somebody was looking for some...",autos,harley davidson
...,...,...,...,...,...,...
19995,7346dx,4Runner,03 JBL Sound System Issue,Just picked up my first 4Runner this week! Lov...,autos,toyota
19996,5ut9e2,4Runner,2nd gen Comes Back From the Dead With No Tools,Hello fellow runners-<lb><lb>I wanted to share...,autos,toyota
19997,66xnwd,4Runner,Tranny cooler in cold temperatures?,So I picked up a 2002 4Runner recently and the...,autos,toyota
19998,8kvxkg,4Runner,Torn between a Wrangler and a 4 Runner...,Ok so I've posted to r/Jeep and figured I shou...,autos,toyota


In [16]:
df = df[df.category == "autos"]
df

,id,subreddit,title,text,category,subcategory
0,8f73s7,Harley,No Club Colors,Funny story. I went to college in Las Vegas. T...,autos,harley davidson
1,4z6vee,Harley,Mystery Rattle,I have a stock 2010 FXD that has a rate I can'...,autos,harley davidson
2,6x4tdk,Harley,Gum on tailpipe,So I'm on a bike trip with my father and we st...,autos,harley davidson
3,7l7rer,Harley,Changing up the bars!,So I’m gonna change out my T bars that came on...,autos,harley davidson
4,683pc7,Harley,Spent 2 days and made some Mini Floorboards fo...,"A few weeks ago, somebody was looking for some...",autos,harley davidson
...,...,...,...,...,...,...
19995,7346dx,4Runner,03 JBL Sound System Issue,Just picked up my first 4Runner this week! Lov...,autos,toyota
19996,5ut9e2,4Runner,2nd gen Comes Back From the Dead With No Tools,Hello fellow runners-<lb><lb>I wanted to share...,autos,toyota
19997,66xnwd,4Runner,Tranny cooler in cold temperatures?,So I picked up a 2002 4Runner recently and the...,autos,toyota
19998,8kvxkg,4Runner,Torn between a Wrangler and a 4 Runner...,Ok so I've posted to r/Jeep and figured I shou...,autos,toyota


### 1.2 Save the data

In [17]:
df.to_pickle(os.path.join(data_path, "reddit-selfposts/reddit_dataframe.pkl"))

In [18]:
db_name = "reddit-selfposts.db"
con = sqlite3.connect(os.path.join(data_path, "reddit-selfposts", db_name))
df.to_sql("posts", con, index=False, if_exists="replace")
con.close()

In [19]:
con = sqlite3.connect(os.path.join(data_path, "reddit-selfposts", db_name))
pd.read_sql("SELECT * FROM posts", con)
con.close()

## 2. Cleaning text data

In [20]:
df.text

0        Funny story. I went to college in Las Vegas. T...
1        I have a stock 2010 FXD that has a rate I can'...
2        So I'm on a bike trip with my father and we st...
3        So I’m gonna change out my T bars that came on...
4        A few weeks ago, somebody was looking for some...
                               ...                        
19995    Just picked up my first 4Runner this week! Lov...
19996    Hello fellow runners-<lb><lb>I wanted to share...
19997    So I picked up a 2002 4Runner recently and the...
19998    Ok so I've posted to r/Jeep and figured I shou...
19999    I read a lot Forums and people recommend getti...
Name: text, Length: 20000, dtype: object

### 2.1 Blueprint: Identify noise with regular expressions

In [21]:
RE_SUSPICIOUS = re.compile(r'[&#<>{}\[\]\\]')

In [22]:
def impurity(text, min_len=10):
    """
    returns the sahre of suspicious characters in a text
    """
    if text == None or len(text) < min_len:
        return 0
    else:
        return len(RE_SUSPICIOUS.findall(text))/len(text)

In [23]:
df['impurity'] = df['text'].apply(impurity, min_len=10)

In [24]:
df[["text", "impurity"]].sort_values(by="impurity", ascending=False).head(3)

,text,impurity
14983,Looking at buying a 335i with 39k miles and 11...,0.214716
8630,I'm looking to lease an a4 premium plus automa...,0.165099
15141,Breakdown below:<lb><lb>Elantra GT<lb><lb>2.0L...,0.139130


In [25]:
count_words(df, column='text', preprocess=lambda t: re.findall(r'<[\w/]*>', t))

,freq
token,
<lb>,100729
<tab>,642


### 2.2 Blueprint: Removing noise with regular expressions

In [49]:
def clean(text):
    # convert html escapes like &amp; to characters
    text = html.unescape(text)
    # tags like <tab>
    text = re.sub(r"<[^<>]*>", " ", text)
    # markdown URLs like [Some text](https://....)
    text = re.sub(r"\[([^\[\]]*)\]\([^\(\)]*\)", r"\1", text)
    # text or code in brackets like [0]
    text = re.sub(r"\[[^\[\]]*\]", " ", text)
    # standalone sequences of hyphens like --- or ==
    text = re.sub(r"(?:^|\s)[\-=\+]{2,}(?:\s|$)", " ", text)
    # sequences of white spaces
    text = re.sub(r"\s+", " ", text)
    return text.strip()

In [52]:
df["clean_text"] = df.text.apply(clean)

In [55]:
df["impurity"] = df.clean_text.apply(impurity, min_len=20)

In [57]:
df[["clean_text", "impurity"]].sort_values(by="impurity", ascending=False)

,clean_text,impurity
15361,Split b/w 2 genesis options. Hyundai Genesis\ ...,0.039867
1714,"Mustang 2018, 2019, or 2020? Must Haves!! 1. H...",0.031980
17941,"Just like the title says, when I am manually s...",0.028914
16959,"At the dealership, they offered an option for ...",0.026455
12828,"I am looking at four Caymans, all are in a sim...",0.024631
...,...,...
6866,Im lookin to get a 2008 honda fit with 150000k...,0.000000
6865,My car when I'm driving and know I'm going to ...,0.000000
6864,"Hey /r/Honda, Ever since upgrading to iOS 11 a...",0.000000
6863,I just bought a 97 civic with 105k miles on it...,0.000000
